# Train a Model for Detecting Surface Water

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_water_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [2]:
 %pip install geoai-py

## Import libraries

In [3]:
import geoai

## Download sample data

In [4]:
train_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip/naip_water_train.tif"
train_masks_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip/naip_water_masks.tif"
test_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip/naip_water_test.tif"

In [5]:
train_raster_path = geoai.download_file(train_raster_url)
train_masks_path = geoai.download_file(train_masks_url)
test_raster_path = geoai.download_file(test_raster_url)

naip_water_train.tif: 100%|██████████| 150M/150M [00:03<00:00, 41.5MB/s]
naip_water_masks.tif: 100%|██████████| 421k/421k [00:00<00:00, 9.04MB/s]
naip_water_test.tif: 100%|██████████| 158M/158M [00:02<00:00, 64.0MB/s]


In [6]:
geoai.print_raster_info(train_raster_path, show_preview=False)

===== RASTER INFORMATION: naip_water_train.tif =====
Driver: GTiff
Dimensions: 5390 x 7580 pixels
Number of bands: 4
Data type: uint8
Coordinate Reference System: EPSG:26914
Georeferenced Bounds: BoundingBox(left=485430.0, bottom=5204860.0, right=490820.0, top=5212440.0)
Pixel Resolution: 1.0, 1.0
NoData Value: None

----- Band Statistics -----
Band 1:
  Min: 36.00
  Max: 238.00
  Mean: 127.01
  Std Dev: 34.38
Band 2:
  Min: 46.00
  Max: 233.00
  Mean: 121.56
  Std Dev: 22.03
Band 3:
  Min: 57.00
  Max: 237.00
  Mean: 105.59
  Std Dev: 18.37
Band 4:
  Min: 19.00
  Max: 238.00
  Mean: 135.77
  Std Dev: 36.26


## Visualize sample data

In [7]:
geoai.view_raster(train_masks_url, nodata=0, basemap=train_raster_url)

In [8]:
geoai.view_raster(test_raster_url)

## Create training data

In [9]:
out_folder = "output"

In [ ]:
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_masks_path,
    tile_size=512,
    stride=128,
    buffer_radius=0,
)

Detected in_class_data as raster: naip_water_masks.tif
Raster CRS: EPSG:26914
Raster dimensions: 5420 x 7602

Raster info for naip_water_train.tif:
  CRS: EPSG:26914
  Dimensions: 5390 x 7580
  Resolution: (1.0, 1.0)
  Bands: 4
  Bounds: BoundingBox(left=485430.0, bottom=5204860.0, right=490820.0, top=5212440.0)
Found 1 unique classes in raster: [1]


Generated: 1398, With features: 1323:  61%|██████▏   | 1398/2280 [02:43<02:06,  6.98it/s]

## Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=4,
    pretrained=True,
    batch_size=4,
    num_epochs=10,
    learning_rate=0.005,
    val_split=0.2,
)

## Run inference

In [ ]:
masks_path = "naip_water_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=128,
    confidence_threshold=0.3,
    batch_size=4,
    num_channels=4,
)

## Vectorize masks

In [ ]:
output_path = "naip_water_prediction.geojson"
gdf = geoai.raster_to_vector(
    masks_path, output_path, min_area=1000, simplify_tolerance=1
)

In [ ]:
gdf = geoai.add_geometric_properties(gdf)

In [ ]:
len(gdf)

In [ ]:
geoai.view_vector_interactive(gdf, tiles=test_raster_url)

In [ ]:
gdf["elongation"].hist()

In [ ]:
gdf_filtered = gdf[gdf["elongation"] < 10]

In [ ]:
len(gdf_filtered)

## Visualize results

In [ ]:
geoai.view_vector_interactive(gdf_filtered, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=gdf_filtered,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/a269b5a0-9f72-4ed8-8b2d-a175bbc45a23)